# Key Concepts

- **`gr.Interface`** → The simplest way to connect a Python function to a UI.
- **Inputs** (`gr.Textbox`, `gr.Image`, `gr.Slider`,`gr.Button`,etc.)
- **Outputs** (`gr.Textbox`, `gr.Image`, etc.)
- **`launch()`** → Runs the app locally.

In [ ]:
import gradio as gr
def welcome(name):
    return f"hello my name is {name}"

demo = gr.Interface(
    fn=welcome,
    inputs=gr.Textbox(label="Enter something"),
    outputs=gr.Textbox(label="Response")
)

demo.launch()

In [ ]:
import gradio as gr

def welcome(name, age):
    return f"Hello! My name is {name} and I am {age} years old."

demo = gr.Interface(
    fn=welcome,
    inputs=[
        gr.Textbox(label="Name"),
        gr.Slider(label="Age", value=1, minimum=1, maximum=100, step=1)
    ],
    outputs=gr.Textbox(label="Response")
)

demo.launch()

# Multi-Turn Chat (Chatbot Mode)

**Goal:** Create a **persistent conversation** inside the app (like ChatGPT's scrollable chat area).

## Key Concepts

- **`gr.Chatbot`** → Special component to store and display chat messages.
- **Session State** (`history`) → Stores chat so it doesn't reset after each message.
- **Updating UI dynamically** using return values.

In [ ]:
import gradio as gr

def chatbot_response(message, history):
    # Append user message
    history.append(("User: " + message, "Bot: " + message.upper()))
    return history, history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your message")
    state = gr.State([])  # Store chat history

    def respond(user_message, history):
        history.append((user_message, "Echo: " + user_message))
        return history, history

    msg.submit(respond, [msg, state], [chatbot, state])

demo.launch()


# Image Upload + Chat

**Goal:** Let the user **upload an image**, send it to the bot, and get a response.

## Key Concepts

- **`gr.Image()`** as input
- **Handling both text and images** in chat
- **Showing uploaded images** in `gr.Chatbot`

In [ ]:
import gradio as gr

def image_chat(user_message, user_image, history):
    # Add image message
    if user_image is not None:
        history.append((user_message, "You sent an image!"))
    else:
        history.append((user_message, "Echo: " + user_message))
    return history, history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your message")
    img = gr.Image(type="filepath", label="Upload image (optional)")
    state = gr.State([])

    send_btn = gr.Button("Send")

    send_btn.click(image_chat, [msg, img, state], [chatbot, state])

demo.launch()


# Persistent Chat Like ChatGPT

**Goal:** Make **previous chats retrievable** even after page reload (like ChatGPT's left-hand sidebar).

## Key Concepts

- **Saving chats to a file or database**
- **Loading chats at startup**
- **Dynamic components** (loading old chats into chatbot window)

In [ ]:
import gradio as gr
import json
import os

CHAT_FILE = "chat_history.json"

def load_history():
    if os.path.exists(CHAT_FILE):
        with open(CHAT_FILE, "r") as f:
            return json.load(f)
    return []

def save_history(history):
    with open(CHAT_FILE, "w") as f:
        json.dump(history, f)

def chatbot_response(message, history):
    history.append((message, "Echo: " + message))
    save_history(history)
    return history, history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Type here")
    state = gr.State(load_history())

    msg.submit(chatbot_response, [msg, state], [chatbot, state])

demo.launch()
